# Tratamento de Dados com pandas e numpy

### Este notebook realiza o pré - processamento de um conjunto de dados de clientes, com foco em limpeza, anonimização e padronização.

## 1. Inicialização

Bibliotecas utilizadas:
- `pandas` - para manipulação de dados tabulares
- `numpy` - para operações numéricas e tratamento de valores nulos

- `numpy` - serve para manipulação de arrays e operações matemáticas de alto desempenho e serve como base para diversas outras bibliotecas como o pandas, SciPy, scikit-learn. Os principais recursos do NumPy são:
> - (`ndarray`) - estrutura central do NumPy, mais eficiente que listas nativas do Python para operações numéricas.
> - Operações vetorizadas - que permite aplicar funções matemáticas diretamente sobre arrays, sem necessidade de loops explícitos.
> - Funções matemáticas e estatísticas como: `np.mean()`, `np.std()`, `np.sum()`, `np.median()`, `np.percentile()`
> - Tratamento de valores nulos e substituições: `np.nan`, `np.where()`, `np.isnan()`
> - Geração de números aleatórios: `np.random.rand()`, `np.random.randint()`, `np.random.normal()`
> - Manipulação de arrays: `reshape()`, `flatten()`, `concatenate()`, `split()`

In [ ]:
import pandas as pd
import numpy as np

#### Configurações iniciais:

Essas opções garantem que o conteúdo das colunas seja exibido por completo no terminal ou notebook.

In [ ]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None

df = pd.read_csv('../data/clientes_remocao_outliers.csv')

print(df.head())

## 2. Mascarando os dados pessoais

- CPF mascarado
- Protege a privacidade dos clientes mantendo apenas parte do CPF visível.

In [ ]:
df['cpf_mascara'] = df['cpf'].apply(lambda cpf: f'{cpf[:3]}.***.***-{cpf[-2:]}')

## 3. Correção de datas e cálculo de idade

- Conversão de datas com `pd.to_datetime()`
- Substituição de datas futuras por uma  data padrão (1900 - 01 - 01)
- Cálculo da idade ajustada com verificação de mês e dia
- Remoção de idades acima de 100 anos como possível erro

In [ ]:
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')

data_atual = pd.to_datetime('today')
df['data_atualizada'] = df['data'].where(df['data'] <= data_atual, pd.to_datetime('1900-01-01'))
df['idade_ajustada'] = data_atual.year - df['data_atualizada'].dt.year
df['idade_ajustada'] -= ((data_atual.month <= df['data_atualizada'].dt.month) & (data_atual.day < df['data_atualizada'].dt.day)).astype(int)
df.loc[df['idade_ajustada'] > 100, 'idade_ajustada'] = np.nan

## 4. Padronização de endereço

- Extração de endereço curto, bairro e estado a partir de campos multilineares
- Validação de comprimento do endereço
- Normalização da sigla do estado com base em uma tupla de estados válidos.

In [ ]:
df['endereco_curto'] = df['endereco'].apply(lambda x: x.split('\n')[0].strip())
df['bairro'] = df['endereco'].apply(lambda x: x.split('\n')[1].strip() if len(x.split('\n')) > 1 else 'Desconhecido')
df['estado_sigla'] = df['endereco'].apply(lambda x: x.split(' / ')[-1].strip() if len(x.split('\n')) > 1 else 'Desconhecido')

df['endereco_curto'] = df['endereco_curto'].apply(lambda x: 'Endereço inválido' if len(x) > 50 or len(x) < 5 else x)


## 5. Correção de dados inválidos

- CPF com tamanho diferente de 14 caracteres é marcado como 'CPF inválido'
- Estados não reconhecidos são rotulados como 'Desconhecido'

In [ ]:
df['cpf'] = df['cpf'].apply(lambda x: x if len(x) == 14 else 'CPF inválido')

estados_br = ('AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA',
              'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO')
df['estado_sigla'] = df['estado_sigla'].str.upper().apply(lambda x: x if x in estados_br else 'Desconhecido')

print('Dados tratados:\n', df.head())

## 6. Exportação do DataFrame tratado

In [ ]:
df['cpf'] = df['cpf_mascara']
df['idade'] = df['idade_ajustada']
df['endereco'] = df['endereco_curto']
df['estado'] = df['estado_sigla']
df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'bairro', 'estado']]

df_salvar.to_csv('clientes_tratados.csv', index=False)

print('Novo DataFrame: \n', pd.read_csv('../data/clientes_tratados.csv'))